# jensen shannon main process

In [1]:
import re 
import pandas as pd
import numpy as np

def H_p(arr):
   
    ret = -np.nansum(np.log2(arr)*arr)
    return ret
def E_norm(arr):
    return np.log2(arr + 1)/np.sum(np.log2(arr + 1),axis = 1, keepdims=True)
fr = open(r"D:\数据\GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct\GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct","r")
ll = fr.readlines()

index_list = []
for i in range(3,len(ll)):
    tmp = re.split("\t",ll[i].strip())[0]
    tmp1 = re.split("\.",tmp)[0]
    index_list.append(tmp1)

column_list = re.split("\t",ll[2].strip())[2:]
##

pre_np_list = []
for i in range(3,len(ll)):
    tmp_list = []
    tmp = re.split("\t",ll[i].strip())[2:]
    for l in tmp:
        tmp_list.append(float(l))
    pre_np_list.append(tmp_list)

raw_matrix = np.array(pre_np_list)
divider = np.sum(raw_matrix,axis = 1, keepdims = True)
#e_norm = raw_matrix/divider
e_norm = E_norm(raw_matrix) #log2 norm

print(np.max(e_norm),np.min(e_norm))
js = np.zeros((np.shape(e_norm)[0],np.shape(e_norm)[1]))
for i in range(np.shape(e_norm)[0]):
    Hp1 = H_p(e_norm[i])
    tmp = e_norm[i]
    for j in range(np.shape(e_norm)[1]):
        tmp2 = np.zeros(np.shape(e_norm)[1])
        tmp2[j] = tmp2[j] + 1
        hp12 = H_p((tmp + tmp2)/2)
        js[i][j] = hp12 - Hp1/2

js_sqrt = np.sqrt(js)
js_sp = 1-js_sqrt
print(js_sp.shape,np.max(js_sp),np.min(js_sp))

#写入csv
if len(column_list) == js_sp.shape[1] and len(index_list) == js_sp.shape[0]:
    df1 = pd.DataFrame(js_sp, columns=column_list ,index=index_list)
    df1.to_csv(r"D:\数据\GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct\gtex_js.csv",index = False)


#  Graphene tissue top300 and bottom 1000 files generation of 202 diseases

In [ ]:
import numpy as np
import pandas as pd
import json
import re

df1 = pd.read_csv(r"D:\数据\GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct\gtex_js.csv")
df1.index = df1['Unnamed: 0'].values
df2 = pd.read_csv(r"../network_data/sy_ent_ens.csv")
# map dict
fj = open(r"../network_data/map.json", 'r')
json_map = json.load(fj)
# print(json_map[0])
map_dict = json_map[0]
#
fr = open(r"../disease gene reprioritization/gwas_cui_MAPPED_TRAIT_threshold_30_tab.txt")
llr = fr.readlines()
arr = np.load(r"../disease gene reprioritization/result/logits202.npy")
for i in range(len(arr)):
    # handle disease name
    dis_name = re.split("\t", llr[i].strip())[1]
    #
    dis = arr[i]
    dis_dict = dict(zip(range(18547), dis))
    or_dis_dict = sorted(dis_dict.items(), key=lambda d: d[1], reverse=True)
    # or_dis_dict_2 = sorted(dis_dict.items(), key= lambda d:d[1], reverse= False)
    # print(or_dis_dict[:100])
    selected_L_top = or_dis_dict[:300]
    selected_L_bottom = or_dis_dict[-1000:]

    #     fw_top = open(r"D:\R_project\fig3\tissue_all\psoriasis_top300.txt", "w")
    #     fw_bottom = open(r"D:\R_project\fig3\tissue_all\squamous cell lung carcinoma_bottom1000.txt", "w")
    #     df = pd.read_csv(r"E:\projects\set2gaussian-master\src\sy_ent_ens.csv")
    ret_list_top = []
    for item in selected_L_top:
        ensg = df2[df2["entrez_id"].astype(str) == map_dict[str(item[0])]]["ensembl_gene_id"].values
        if ensg.size > 0:
            ensg = str(ensg[0])
            try:
                if len(df1.loc[ensg]) == 55:
                    ret_list_top.append(df1.loc[ensg])

            except:
                continue
    df3 = pd.DataFrame(ret_list_top)  # 注意dataframe的用法这里
    df3.to_csv("D:\\R_project\\fig3\\tissue_all\\" + dis_name + "_top300.csv", index=False)
    #fw_top.close()
    #fw_bottom.close()
    ret_list_bottom = []
    for item in selected_L_bottom:
        ensg = df2[df2["entrez_id"].astype(str) == map_dict[str(item[0])]]["ensembl_gene_id"].values
        if ensg.size > 0:
            ensg = str(ensg[0])
            try:
                if len(df1.loc[ensg]) == 55:
                    ret_list_bottom.append(df1.loc[ensg])

            except:
                continue
    df3 = pd.DataFrame(ret_list_bottom)  # 注意dataframe的用法这里
    df3.to_csv("D:\\R_project\\fig3\\tissue_all\\" + dis_name + "_bottom1000.csv", index=False)
    print(i)

# #  GWAS tissue top300 and bottom 1000 files generation of 202 diseases

In [5]:
import numpy as np
import pandas as pd
import json
import re
import os
import math
from scipy import stats

dis_pair = os.listdir(r"D:\R_project\fig3\tissue_gwas")
heat_map_list = []

for i in range(math.floor(len(dis_pair)/2)):
#for i in range(0,2):
    tissue_list = []
    print(i)
    print(dis_pair[2*i])
    df1 = pd.read_csv("D:\\R_project\\fig3\\tissue_gwas\\" + dis_pair[2*i +1])
    print(dis_pair[2*i+1])
    df2 = pd.read_csv("D:\\R_project\\fig3\\tissue_gwas\\" + dis_pair[2*i])
    dis_name = re.sub("_topGwas.csv", "", dis_pair[2*i +1])
    for tissue in df1.columns.tolist():
        if tissue not in ["Unnamed: 0","Testis"]:
            x1 = df1[tissue].tolist()
            x2 = df2[tissue].tolist()
            res = stats.mannwhitneyu(x1 ,x2)
            p_value = res.pvalue
            tissue_list.append({"tissue_name":tissue,"dis":dis_name,"p_value":p_value})
    df3 = pd.DataFrame(tissue_list)
    df3.to_csv("D:\\R_project\\fig3\\js_gwas\\" + "js_pvalue_" + dis_name +".csv", index = False)
    heat_map_list.extend(tissue_list)
########################
df4 = pd.DataFrame(heat_map_list)
df4.to_csv(r"D:\R_project\fig3\js_gwas\js_pvalue_gwas_201.csv", index = False)

0
Abnormality of chromosome segregation_bottom1000.csv
Abnormality of chromosome segregation_topGwas.csv
1
Abnormality of refraction_bottom1000.csv
Abnormality of refraction_topGwas.csv
2
adiponectin measurement_bottom1000.csv
adiponectin measurement_topGwas.csv
3
adolescent idiopathic scoliosis_bottom1000.csv
adolescent idiopathic scoliosis_topGwas.csv
4
adult onset asthma_bottom1000.csv
adult onset asthma_topGwas.csv
5
age at menarche_bottom1000.csv
age at menarche_topGwas.csv
6
age at menopause_bottom1000.csv
age at menopause_topGwas.csv
7
aging_bottom1000.csv
aging_topGwas.csv
8
albuminuria_bottom1000.csv
albuminuria_topGwas.csv
9
alcohol drinking_bottom1000.csv
alcohol drinking_topGwas.csv
10
alkaline phosphatase measurement_bottom1000.csv
alkaline phosphatase measurement_topGwas.csv
11
alopecia areata_bottom1000.csv
alopecia areata_topGwas.csv
12
Alopecia_bottom1000.csv
Alopecia_topGwas.csv
13
Alzheimer's disease_bottom1000.csv
Alzheimer's disease_topGwas.csv
14
amyotrophic later

114
leptin measurement_bottom1000.csv
leptin measurement_topGwas.csv
115
leukocyte count_bottom1000.csv
leukocyte count_topGwas.csv
116
longevity_bottom1000.csv
longevity_topGwas.csv
117
low density lipoprotein cholesterol measurement_bottom1000.csv
low density lipoprotein cholesterol measurement_topGwas.csv
118
magnesium measurement_bottom1000.csv
magnesium measurement_topGwas.csv
119
malaria_bottom1000.csv
malaria_topGwas.csv
120
mathematical ability_bottom1000.csv
mathematical ability_topGwas.csv
121
melanoma_bottom1000.csv
melanoma_topGwas.csv
122
memory performance_bottom1000.csv
memory performance_topGwas.csv
123
motion sickness_bottom1000.csv
motion sickness_topGwas.csv
124
Moyamoya disease_bottom1000.csv
Moyamoya disease_topGwas.csv
125
mucocutaneous lymph node syndrome_bottom1000.csv
mucocutaneous lymph node syndrome_topGwas.csv
126
multiple myeloma_bottom1000.csv
multiple myeloma_topGwas.csv
127
multiple sclerosis_bottom1000.csv
multiple sclerosis_topGwas.csv
128
myocardial i

# GWAS matrix for heatmap

In [6]:
import numpy as np
import pandas as pd
import json
import re
import os
import math

df_mat = pd.DataFrame()

# change to "D:\\R_project\\fig3\\tissue_all\\" dir to compute graphene matrix
dis_pair = os.listdir(r"D:\R_project\fig3\js_gwas") 
df0 =  pd.read_csv("D:\\R_project\\fig3\\js_gwas\\" + dis_pair[0])
ind_name = df0["tissue_name"].tolist()
# print(ind_name)
for i in range(len(dis_pair)):
    df1 = pd.read_csv("D:\\R_project\\fig3\\js_gwas\\" + dis_pair[i])
    df_mat[df1["dis"][0]] = df1["p_value"]
df_mat.index = ind_name
df_mat.to_csv(r"D:\R_project\fig3\js_value_mat_gwas.csv")